In [1]:
import os

print(os.getcwd())

os.chdir('../')

print(os.getcwd())

c:\Users\Muhammed Nishan\OneDrive - Logic Plum, Inc\Documents\Learnings\MLOps Projects\datascienceproject\research
c:\Users\Muhammed Nishan\OneDrive - Logic Plum, Inc\Documents\Learnings\MLOps Projects\datascienceproject


In [2]:
# Reading the csv file
import pandas as pd

data = pd.read_csv('artifacts/data_ingestion/winequality-red.csv')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
# Creating Data Transformation Config

from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [12]:
# Updating Configuration Manager

from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

class ConfigurationManager:
	def __init__(self, 
		config_filepath = CONFIG_FILE_PATH,
		params_filepath = PARAMS_FILE_PATH,
		schema_filepath = SCHEMA_FILE_PATH):
		
		# Loading configs, params and schema
		self.config = read_yaml(config_filepath)
		self.params = read_yaml(params_filepath)
		self.schema = read_yaml(schema_filepath)

		# Creating Artifacts
		create_directories([self.config.artifacts_root])
	

	def get_data_transformation_config(self) -> DataTransformationConfig:
		config = self.config.data_transformation

		create_directories([config.root_dir])

		data_transformation_config = DataTransformationConfig(
			root_dir = config.root_dir,
			data_path = config.data_path
		)

		return data_transformation_config

In [16]:
# Components - Data Transformation

import os
from src.datascience import logger 
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:

    def __init__(self, config: DataTransformationConfig):
        self.config = config
    
    def train_test_split(self):

        data = pd.read_csv(self.config.data_path)

        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, 'train.csv'), index=False)
        test.to_csv(os.path.join(self.config.root_dir, 'test.csv'), index=False)

        logger.info("Splitted data into training and testing sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print("Training Dataset Shape:", train.shape)
        print("Testing Dataset Shape:", test.shape)

In [17]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_split()
except Exception as e:
    raise e

[2025-07-23 18:28:08,990: INFO: common: YAML File: config\config.yaml Loaded Successfully.]
[2025-07-23 18:28:08,993: INFO: common: YAML File: params.yaml Loaded Successfully.]
[2025-07-23 18:28:08,996: INFO: common: YAML File: schema.yaml Loaded Successfully.]
[2025-07-23 18:28:08,997: INFO: common: Created Directory at artifacts]
[2025-07-23 18:28:08,998: INFO: common: Created Directory at artifacts/data_transformation]
[2025-07-23 18:28:09,026: INFO: 3098282009: Splitted data into training and testing sets]
[2025-07-23 18:28:09,029: INFO: 3098282009: (1199, 12)]
[2025-07-23 18:28:09,031: INFO: 3098282009: (400, 12)]
Training Dataset Shape: (1199, 12)
Testing Dataset Shape: (400, 12)
